In [1]:
!pip install nltk

In [1]:
import random
import json
import pickle
import numpy as np

import nltk
from nltk.stem import WordNetLemmatizer

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD


In [2]:
lemmatizer = WordNetLemmatizer()

intents = json.loads(open('/content/sample_data/intents.json').read())

words = []
classes = []
documents = []
ignore_letters = ['?', '!',',','.']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list,intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words = sorted(set(words))

classes = sorted(set(classes))

pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))


In [3]:
training = []
output_empty = [0] * len(classes)

for document in documents:
    bag =[]
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1
    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training)

train_x = list(training[:, 0])
train_y = list(training[:, 1])

<ipython-input-3-037706e4c13f>:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [4]:
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers.schedules import ExponentialDecay

model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

initial_learning_rate = 0.01
lr_schedule = ExponentialDecay(
    initial_learning_rate, decay_steps=100000, decay_rate=0.96, staircase=True
)

# Use SGD with the learning rate schedule
sgd = SGD(learning_rate=lr_schedule, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbotmodel.h5', hist)

print('Done')

Epoch 1/200
10/10 [==============================] - 2s 4ms/step - loss: 2.3449 - accuracy: 0.1489
Epoch 2/200
10/10 [==============================] - 0s 4ms/step - loss: 2.2483 - accuracy: 0.2553
Epoch 3/200
10/10 [==============================] - 0s 4ms/step - loss: 2.1851 - accuracy: 0.1915
Epoch 4/200
10/10 [==============================] - 0s 4ms/step - loss: 2.1295 - accuracy: 0.2979
Epoch 5/200
10/10 [==============================] - 0s 4ms/step - loss: 2.1214 - accuracy: 0.2979
Epoch 6/200
10/10 [==============================] - 0s 4ms/step - loss: 2.1066 - accuracy: 0.2553
Epoch 7/200
10/10 [==============================] - 0s 3ms/step - loss: 2.0147 - accuracy: 0.2553
Epoch 8/200
10/10 [==============================] - 0s 4ms/step - loss: 1.9282 - accuracy: 0.4043
Epoch 9/200
10/10 [==============================] - 0s 4ms/step - loss: 1.7845 - accuracy: 0.3830
Epoch 10/200
10/10 [==============================] - 0s 4ms/step - loss: 1.7523 - accuracy: 0.4043
Epoch 11/

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [5]:
import random
import json
import pickle
import numpy as np

import nltk
from nltk.stem import WordNetLemmatizer

from tensorflow.keras.models import load_model


lemmatizer = WordNetLemmatizer()

In [6]:
intents = json.loads(open('/content/sample_data/intents.json').read())
words = pickle.load(open('/content/words.pkl', 'rb'))
classes = pickle.load(open('/content/classes.pkl', 'rb'))
model = load_model('/content/chatbotmodel.h5')


In [7]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word)  for word in sentence_words]
    return sentence_words

def bag_of_words(sentence):
    sentence_words= clean_up_sentence(sentence)
    bag = [0] * len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
            if word == w:
                bag[i] = 1

    return np.array(bag)

def predict_class(sentence):
    bow = bag_of_words(sentence)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]

    results.sort(key=lambda  x:x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({'intent': classes[r[0]], 'probability': str(r[1])})
    return return_list


def get_response(intents_list,intents_json):
    tag= intents_list[0]['intent']
    list_of_intents =intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    return result

In [8]:
while True:
    message = input("| You: ")
    if message == "bye" or message == "Goodbye":
        ints = predict_class(message)
        res = get_response(ints, intents)
        print("| Bot:", res)
        exit()

    else:
        ints = predict_class(message)
        res = get_response(ints, intents)
        print("| Bot:", res)


| You: chao
1/1 [==============================] - 0s 81ms/step
| Bot: lau ngay khong gap, nhin ban map len nhi
| You: hello
1/1 [==============================] - 0s 17ms/step
| Bot: lau ngay khong gap, nhin ban map len nhi
| You: toi can ban giup
1/1 [==============================] - 0s 17ms/step
| Bot: toi co the giup gi cho ban
| You: kết thúc
1/1 [==============================] - 0s 31ms/step
| Bot: Hello!
| You: bye
1/1 [==============================] - 0s 19ms/step
| Bot: Sad to see you go :(
|===================== The Program End here! =====================|


KeyboardInterrupt: Interrupted by user